## 特征选择和稀疏学习

在学习器的训练中，可能会有很多的冗余的特征。那么怎么提取关键的特征那？

### 1.1 子集搜索与评价
子集搜索（subset search）与子集评价(subset evaluation)。将子集搜索和子集评价两个环节结合起来，就可以得到特征选择的方法。如向前搜索与信息熵结合，这显然与决策树算法非常相似。

#### 子集搜索环节

- forward 向前搜索，一次增加一个特征，然后再训练。
- backward 向后搜索，一次减少一个特征，然后再训练。
- bidirectional 双向搜索.

特征子集搜索使用的都是贪心算法，会因为穷举而出现组合爆炸的情况。

#### 子集评价环节
可以使用信息熵和信息增益来对特征进行评估。选择信息最大的特征。

常见的特征选择方法可分为三类：过滤式(filter)、包裹式(wrapper) 和嵌入式(embedding)。

### 1.2 过滤式选择

过滤式方法是先进行特征选择，然后再进行学习器的训练。特征选择与后续的学习器训练过滤。

Relief（Relevant Features）是一种著名的过滤式特征选择方法，该方法设计一个『相关统计量』来度量特征的重要性。

给定训练集: $\{(\textbf{x}_1, y_1), (\textbf{x}_2, y_2), ... ,(\textbf{x}_m, y_m)\}$, 对于每个样本 $\textbf{x}_i$, Relief 先在同类样本(如同是正样本) 找其最近的 $\textbf{x}_{i, nh}$ 称为 "猜中近邻"（near-hit）, 然后再从与  $\textbf{x}_i$ 相异的异类样本如(如负类样本）中寻找其最近邻  $\textbf{x}_i,nm$,称为『猜错近邻』(near-miss)，然后计算统计量对应特征 j 的分量为：

$$
\delta^j = \sum_{u}-diff(x_i^j, x^j_{i,nh})^2 + diff(x_i^j, x^j_{i,nm})^2
$$

如果 $\delta^j  > 0$  则说明属性 j 可以用来区分同类与异类样本；相反则说明属性 j 不适合用来区分样本。我们可以选择 $\delta^j $ 最小的几个作为特征。

### 1.3 包裹式选择

包裹式选择是特征子集的搜索过程，搜索后就用于学习器的训练。其训练出的学习器效果要好于过滤式，但是计算量会比较耗时。如 Las Vegas Wrapper.

### 1.4 嵌入式选择与 L1 正则

与前两者特征选择和学习器有明显的阶段区分不同，嵌入式特征选择是将特征选择过程与学习器训练过程融为一体，两者在同一个优化过程中完成，即在学习器训练过程中自动地进行了特征选择。

在线性模型中，我们经常引入正则化项来防止过拟合。即如下式：

$$
\min_{\textbf{w}} \sum_{i = 1}^{m}(y_i - \textbf{w}^T \textbf{x}_i)^2 + \frac{\lambda}{2}||\textbf{w}||^p
$$

当 p = 1, p = 2 时分别称为 L1 正则和 L2 正则。L1 正则和 L2 正则都有助于降低过拟合风险，但是 L1 还会带一个额外的好处：它比后者更容易获得 『稀疏』解（sparse）, 即它求得的 $\textbf{w}$ 会有更多的零分量（即更少的非零分量）。这样零分量对应的特性就被过滤掉了。

从周志华老师机器学习 图11.2 和 [什么是正则化](https://charlesliuyx.github.io/2017/10/03/%E3%80%90%E7%9B%B4%E8%A7%82%E8%AF%A6%E8%A7%A3%E3%80%91%E4%BB%80%E4%B9%88%E6%98%AF%E6%AD%A3%E5%88%99%E5%8C%96/) 可以看出当 p = 1 时，正则项的函数『等值线』刚好与损失函数在一个坐标轴上相交，即相对于 L2 来说更容易都求得零点。

于是求解 L1 正则化的结果是得到了仅采用一部分初始化特征的模型，即 L1 就是一种嵌入式特征选择方法，其特征选择过程与学习器训练过程融为一体，同时完成。

不幸的是，L1 正则项有拐点，不是处处可微，给计算带来了难度，很多厉害的论文都是巧妙的使用了一次正则项写出来的，效果十分强大。 L1 正则化问题的求解可以使用近端梯度下降(Proximal Gradient Descent, PGD)。好像 FTRL(Follow The Regularized Leader) 也是使用的是近端梯度下降。




### 1.5 稀疏表示与字典学习
因为 one-hot-encoding 会引入更多的特征，而且会形成稀疏矩阵。SVM 在这样的稀疏矩阵上会得到很好的性能 (周，6.3节和12.4 节)。但是稀疏矩阵的压缩可以用字典学习(codebook learning)的方法来进行，SVD ?

### 1.6 压缩感知(Compress sensing)

压缩感知的思想来自于通信原理。奈奎斯特采样定理，即令采样频率达到模拟信号最高频率的两倍，则采样后的数字信号就保留了模拟信号的全部信息；换言之，由此获得的数字信号能精确重构原模拟信号。

SVD.

我觉得，1.5 和 1.6 都是在用 SVD 来进行特征压缩。

### 总结

1. 为什么我们追求稀疏解？（因为可以过滤特征，减少计算量）。
2. L1 为什么会容易求得稀疏解？
3. L1 在应用梯度下降不是处处可微怎么办？